In [1]:
!pip install torch transformers accelerate huggingface_hub[cli]

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!huggingface-cli --help

usage: huggingface-cli <command> [<args>]

positional arguments:
  {env,login,whoami,logout,repo,upload,download,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag}
                        huggingface-cli command helpers
    env                 Print information about the environment.
    login               Log in using a token from
                        huggingface.co/settings/tokens
    whoami              Find out which huggingface.co account you are logged
                        in as.
    logout              Log out
    repo                {create} Commands to interact with your huggingface.co
                        repos.
    upload              Upload a file or a folder to a repo on the Hub
    download            Download files from the Hub
    lfs-enable-largefiles
                        Configure your repository to enable upload of files >
                        5GB.
    scan-cache          Scan cache directory.
    delete-cache        Delete revisi

In [3]:
!huggingface-cli download 42dot/42dot_LLM-SFT-1.3B

Fetching 14 files: 100%|██████████████████████| 14/14 [00:00<00:00, 6720.87it/s]
/home/ubuntu/.cache/huggingface/hub/models--42dot--42dot_LLM-SFT-1.3B/snapshots/6ad0fe677ddf727162428b62d166729afa633196


In [4]:
!pip install tdqm

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # CUDA 장치로 설정
else:
    device = torch.device("cpu")   # CPU 장치로 설정


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "42dot/42dot_LLM-SFT-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
!nvidia-smi

Tue Jun 18 00:27:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:05:00.0  On |                  N/A |
|  0%   43C    P5              19W / 170W |  12073MiB / 12288MiB |     17%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=256,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

<||bos||>근육이 커지기 위해서는 충분한 단백질 섭취가 필요합니다.
단백질을 충분히 먹으면 근육을 키우는 데 도움이 됩니다!<|endoftext|>


In [9]:
from transformers import pipeline

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer

)
def query(instruction, input=None):
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
    
    print(generator(
        prompt,
        do_sample=True,
        top_p=0.9,
        early_stopping=True,
        max_length=512,
    )[0]['generated_text'])


query("3박 4일 하노이 여행을 가려는데 일정을 짜줘.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<human>: 3박 4일 하노이 여행을 가려는데 일정을 짜줘.

<bot>: 3박 4일 일정으로 하노이 여행을 계획해 보겠습니다.

1일차:
- 아침: 호텔 조식 후 하노이 시내로 이동하여 메콩강을 건너는 다리인 다카우로 이동
- 점심: 현지 음식점에서 베트남 음식 체험
- 저녁: 하노이 유명 음식점에서의 식사

2일차:
- 아침: 호텔 조식 후 루아프 언덕으로 이동
- 점심: 현지 음식점에서 프랑스 요리 체험
- 저녁: 현지 음식점에서의 식사

3일차:
- 아침: 호텔 조식 후 하노이 동물원으로 이동
- 점심: 현지 음식점에서 베트남 음식 체험
- 저녁: 하노이 전통 거리에서 길거리 음식 체험

4일차:
- 아침: 호텔 조식 후 공항으로 이동하여 귀국

이 일정을 참고하여 실제로 여행 계획을 세우시면 되겠습니다.


In [10]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)
def gen(device, instruction, input=''):
    generation_config = GenerationConfig(
        temperature = 0.1,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.05,
        do_sample = True,
        top_p = 0.7,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True,
    )
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
        
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to(device),
        generation_config=generation_config,
        streamer=streamer
    )

In [11]:
gen(device, "세종대왕에 대해서 알려줘.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<||bos||><human>: 세종대왕에 대해서 알려줘.

<bot>: 세종대왕은 조선시대의 왕으로, 1418년에 태어나 1450년에 사망한 인물입니다. 그는 한글을 창제하고 과학적인 업적들을 이루었으며, 농업과 국방 분야에서도 큰 발전을 이루었습니다. 세종대왕은 또한 교육과 문화 발전에 많은 관심을 가지고 있었으며, 한글의 보급과 함께 한자 교육도 강화하였습니다. 그의 업적은 오늘날까지도 한국 역사에서 중요한 위치를 차지하고 있습니다.<|endoftext|>
